In [1]:
import math

from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import scipy
import statsmodels.formula.api as smf
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action='ignore', module='scipy', message='^internal gelsd')

Datset on New York in 2013 found at: https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/table-8/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls

In [2]:
# Read data using pandas
df = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls')

# Delete first three rows
df = df.drop([0,1,2], axis=0)

# Make first row the column headers
df = df.reset_index(drop=True)
df.columns = df.iloc[0]
df = df.drop([0], axis=0)
df = df.reset_index(drop=True)

# Rename all column headers
df.columns = ['City', 'Population', 'Violent Crime', 'Murder and Nonnegligent Manslaughter', 'Rape (revised definition)', 'Rape (legacy definition)', 'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary', 'Larceny-Theft', 'Motor Vehicle Theft', 'Arson']

# Remove entire Unnamed: 4 or 'Rape revised defintion'
df = df.drop('Rape (revised definition)', axis=1)

# Remove null objects
df = df.drop([348, 349, 350], axis=0)
df = df.drop('Arson', axis=1)
df = df.dropna(how='all')

# Convert all columns from object to integer values
df[['Population', 'Violent Crime',
       'Murder and Nonnegligent Manslaughter', 'Rape (legacy definition)',
       'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary',
       'Larceny-Theft', 'Motor Vehicle Theft']] = df[['Population', 'Violent Crime',
       'Murder and Nonnegligent Manslaughter', 'Rape (legacy definition)',
       'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary',
       'Larceny-Theft', 'Motor Vehicle Theft']].astype(int)

# Remove New York and buffalo from data because it skews the data. Although the data is correct
df[df['City'] == 'New York']
df = df[(df['City']!='New York')&(df['City']!='Buffalo')]

# Change name of Property Crime so that it's one word
df['Propertycrime'] = df['Property Crime']
df = df.drop(columns=['Property Crime'])

df['Murder'] = df['Murder and Nonnegligent Manslaughter']
df = df.drop(columns=['Murder and Nonnegligent Manslaughter'])

df['Aggravated_Assault'] = df['Aggravated Assault']
df = df.drop(columns=['Aggravated Assault'])

In [3]:
# Create Features: Population^2, Murder binary, Robbery, binary
# More specifically, create binary categories for murder and robery
df['Population^2'] = df['Population']**2
df['Murder'] = np.where(df['Murder']>0, '1', '0')
df['Robbery_binary'] = np.where(df['Robbery']>0, '1', '0')
#df['Aggravated_Assault'] = np.where(df['Aggravated_Assault']>0, '1', '0')

#Convert new columns to int
df[['Population^2']] = df[['Population^2']].astype(int)
df[['Murder']] = df[['Murder']].astype(int)
df[['Robbery_binary']] = df[['Robbery_binary']].astype(int)
df[['Aggravated_Assault']] = df[['Aggravated_Assault']].astype(int)

In [4]:
# Run Model
regr = linear_model.LinearRegression()
y = df['Propertycrime'].values.reshape(-1, 1)
x = df[['Population','Population^2', 'Murder', 'Robbery_binary']]
regr.fit(x,y)

# Inspect the results
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:\n', regr.score(x,y))


Coefficients: 
 [[1.74484226e-02 7.28298619e-08 1.84108347e+02 6.20590854e+01]]

Intercept: 
 [-35.45330011]

R-squared:
 0.7572767492306005


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [5]:
# Cross Validation test

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=30)
print('With 30% holdout: ' + str(regr.fit(x_train, y_train,).score(x_test, y_test)))
print('Testing on sample:' + str(regr.fit(x,y).score(x,y)))

With 30% holdout: 0.7503983756164383
Testing on sample:0.7572767492306005


In [6]:
# Cross-validation with mutliple folds
from sklearn.model_selection import cross_val_score

cross_val_score(regr, x, y, cv=5)

array([0.81409112, 0.71966237, 0.74405329, 0.61140936, 0.21822869])

In [7]:
# Test for significance in parameters
linear_formula = 'Propertycrime ~ Population+Population^2+Murder+Robbery_binary'

# Fit the model to our data using formula
lm = smf.ols(formula=linear_formula, data=df).fit()

In [8]:
lm.params

Intercept        -80.074
Population         8.359
Population ^ 2    -8.330
Murder           114.402
Robbery_binary   -27.541
dtype: float64

In [9]:
lm.pvalues

Intercept        0.025
Population       0.456
Population ^ 2   0.458
Murder           0.135
Robbery_binary   0.578
dtype: float64

In [10]:
# Applying PLSR
from sklearn.cross_decomposition import PLSRegression

features = df[['Population', 'Burglary', 'Larceny-Theft', 'Motor Vehicle Theft']]
X = features
Y = df['Propertycrime']

pls1 = PLSRegression(n_components=3)

#Fit pls1 to training data
pls1.fit(X,Y)

pls1_pred = pls1.predict(X)

print('R-squared = ', pls1.score(X,Y))



R-squared =  0.9996910610726267


In [11]:
# Cross-validation with mutliple folds
from sklearn.model_selection import cross_val_score

cross_val_score(pls1, X, Y, cv=5)

array([0.99968399, 0.99906992, 0.99862366, 0.99979725, 0.99952227])

In [12]:
# Test for significance in parameters
linear_formula = 'Propertycrime ~ features'

# Fit the model to our data using formula
lm = smf.ols(formula=linear_formula, data=df).fit()

In [13]:
lm.pvalues

Intercept     0.000
features[0]   0.000
features[1]   0.000
features[2]   0.000
features[3]   0.000
dtype: float64

# Test Model on Second Data Set

Dataset on Illinois found at: https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/table-8/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_illinois_by_city_2013.xls

In [14]:
# Process Second Dataset on Illinois

df1 = pd.read_excel('table_8_offenses_known_to_law_enforcement_illinois_by_city_2013.xls')

In [15]:
# Delete first three rows
df1 = df1.drop([0,1,2], axis=0)

# Make first row the column headers
df1 = df1.reset_index(drop=True)
df1.columns = df1.iloc[0]
df1 = df1.drop([0], axis=0)
df1 = df1.reset_index(drop=True)

# Rename all column headers
df1.columns = ['City', 'Population', 'Violent Crime', 'Murder and Nonnegligent Manslaughter', 'Rape (revised definition)', 'Rape (legacy definition)', 'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary', 'Larceny-Theft', 'Motor Vehicle Theft', 'Arson']

# Remove entire Unnamed: 4 or 'Rape revised defintion'
df1 = df1.drop('Rape (revised definition)', axis=1)

# Remove null objects
df1 = df1.drop([77, 398, 506, 507, 508, 509], axis=0)
df1 = df1.drop('Arson', axis=1)
df1 = df1.dropna(how='all')

# Convert all columns from object to integer values
df1[['Population', 'Violent Crime',
       'Murder and Nonnegligent Manslaughter', 'Rape (legacy definition)',
       'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary',
       'Larceny-Theft', 'Motor Vehicle Theft']] = df1[['Population', 'Violent Crime',
       'Murder and Nonnegligent Manslaughter', 'Rape (legacy definition)',
       'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary',
       'Larceny-Theft', 'Motor Vehicle Theft']].astype(int)

# Change name of Property Crime so that it's one word
df1['Propertycrime'] = df1['Property Crime']
df1 = df1.drop(columns=['Property Crime'])

df1['Murder'] = df1['Murder and Nonnegligent Manslaughter']
df1 = df1.drop(columns=['Murder and Nonnegligent Manslaughter'])

df1['Aggravated_Assault'] = df1['Aggravated Assault']
df1 = df1.drop(columns=['Aggravated Assault'])

In [16]:
# Create Features: Population^2, Murder binary, Robbery, binary
# More specifically, create binary categories for murder and robery
df1['Population^2'] = df1['Population']**2
df1['Murder'] = np.where(df1['Murder']>0, '1', '0')
df1['Robbery_binary'] = np.where(df1['Robbery']>0, '1', '0')
#df1['Aggravated_Assault'] = np.where(df1['Aggravated_Assault']>0, '1', '0')

#Convert new columns to int
df1[['Population^2']] = df1[['Population^2']].astype(int)
df1[['Murder']] = df1[['Murder']].astype(int)
df1[['Robbery_binary']] = df1[['Robbery_binary']].astype(int)
df1[['Aggravated_Assault']] = df1[['Aggravated_Assault']].astype(int)

In [17]:
features1 = df1[['Population', 'Burglary', 'Larceny-Theft', 'Motor Vehicle Theft']]
X_ = features1
Y_ = df1['Propertycrime']

In [18]:
#Fit pls1 to training data
pls1_pred = pls1.predict(X_)

print('R-squared = ', pls1.score(X_,Y_))

R-squared =  0.9982728371052765


In [19]:
# Cross-validation with mutliple folds
from sklearn.model_selection import cross_val_score

cross_val_score(pls1, X_, Y_, cv=5)

array([0.99665765, 0.9959959 , 0.99547465, 0.99769769, 0.99874302])

In [20]:
# Test for significance in parameters
linear_formula = 'Propertycrime ~ features1'

# Fit the model to our data using formula
lm = smf.ols(formula=linear_formula, data=df1).fit()

In [21]:
lm.pvalues

Intercept      0.000
features1[0]   0.036
features1[1]   0.000
features1[2]   0.000
features1[3]   0.000
dtype: float64

# Methods applies:

For the first model, I used the features given by the assignment, which included population, population squared, binary robbery and binary murder (whether robbery or murder occured or not). My results were bad as te R squared value was low and inconsistent throughout multiple fold hold-out groups. Furthermore, model was even worse when applied to a different dataset.

In the second model, I applied dimensionality reduction using Partial least Squares. I narrowed down the following features; 'Population', 'Burglary', 'Larceny-Theft' and 'Motor Vehicle Theft' because they had the most significant effects on our model according to T-tests. This model lead to a 99% average accuracy in the r squared value across hold out groups and in the second data set. The improvement from the second model was vast and so I'd discard the initial model which had bad results.

Moving forward, I would review the assumption made about the model, such as heterscedasticity and normality in the residual errors.